In [1]:
import sqlite3
import pandas as pd

# Connect to in-memory SQLite database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create the sales table
cursor.execute("""
CREATE TABLE sales (
    id INTEGER PRIMARY KEY,
    customer_name TEXT,
    product TEXT,
    amount INTEGER,
    date TEXT
);
""")

# Insert example data
sales_data = [
    (1, 'Alice', 'Laptop', 1200, '2024-01-01'),
    (2, 'Bob', 'Phone', 800, '2024-01-02'),
    (3, 'Alice', 'Headphones', 200, '2024-01-03'),
    (4, 'Charlie', 'Laptop', 1300, '2024-01-04'),
    (5, 'Bob', 'Laptop', 1250, '2024-01-05'),
]

cursor.executemany("INSERT INTO sales VALUES (?, ?, ?, ?, ?);", sales_data)
conn.commit()

# 1. Extract all sales by Alice
query1 = "SELECT * FROM sales WHERE customer_name = 'Alice';"
df1 = pd.read_sql_query(query1, conn)
print("Sales by Alice:\n", df1)

# 2. Total sales per customer
query2 = "SELECT customer_name, SUM(amount) AS total_sales FROM sales GROUP BY customer_name;"
df2 = pd.read_sql_query(query2, conn)
print("\nTotal Sales per Customer:\n", df2)

# 3. Top selling product
query3 = """
SELECT product, SUM(amount) AS total_amount
FROM sales
GROUP BY product
ORDER BY total_amount DESC
LIMIT 1;
"""
df3 = pd.read_sql_query(query3, conn)
print("\nTop Selling Product:\n", df3)

# 4. Monthly sales totals
query4 = """
SELECT strftime('%Y-%m', date) AS month, SUM(amount) AS monthly_total
FROM sales
GROUP BY month;
"""
df4 = pd.read_sql_query(query4, conn)
print("\nMonthly Sales Totals:\n", df4)

# Close the connection
conn.close()


Sales by Alice:
    id customer_name     product  amount        date
0   1         Alice      Laptop    1200  2024-01-01
1   3         Alice  Headphones     200  2024-01-03

Total Sales per Customer:
   customer_name  total_sales
0         Alice         1400
1           Bob         2050
2       Charlie         1300

Top Selling Product:
   product  total_amount
0  Laptop          3750

Monthly Sales Totals:
      month  monthly_total
0  2024-01           4750
